# Workflow: Transforming NetCDF raster files to polygon feature class

Author: Kris Su 
Description: \
**WARNING - Workflow described in this notebook requires a polygon boundry layer to be added**, and the process itself uses zonal statistic calculation to find the average measurement inside the polygon boundary.

In [2]:
# import 

import arcpy
import os
import xarray as xr
from arcgis import GIS
from arcgis.features import FeatureLayerCollection
gis = GIS()

In [8]:
# Open target netcdf dataset 
ds = xr.open_dataset(r"C:\Users\Kris\Documents\amazonforcast\data\malaria_amazon\amazon_monthly_ldas\LIS_HIST_2001_to_2023_Monthly.nc")
ds

<xarray.Dataset>
Dimensions:           (time: 276, lat: 540, lon: 660, SoilMoist_profiles: 4,
                       SoilTemp_profiles: 4)
Coordinates:
  * time              (time) datetime64[ns] 2001-01-31 2001-02-28 ... 2023-12-31
  * lat               (lat) float64 -20.98 -20.93 -20.88 ... 5.875 5.925 5.975
  * lon               (lon) float64 -81.97 -81.92 -81.88 ... -49.08 -49.03
Dimensions without coordinates: SoilMoist_profiles, SoilTemp_profiles
Data variables: (12/30)
    Swnet_tavg        (time, lat, lon) float32 ...
    Lwnet_tavg        (time, lat, lon) float32 ...
    Qle_tavg          (time, lat, lon) float32 ...
    Qh_tavg           (time, lat, lon) float32 ...
    Qg_tavg           (time, lat, lon) float32 ...
    Snowf_tavg        (time, lat, lon) float32 ...
    ...                ...
    Qair_f_tavg       (time, lat, lon) float32 ...
    Psurf_f_tavg      (time, lat, lon) float32 ...
    SWdown_f_tavg     (time, lat, lon) float32 ...
    LWdown_f_tavg     (time, lat, lon) float32 ...
    LAI_inst          (time, lat, lon) float32 ...
    Greenness_inst    (time, lat, lon) float32 ...
Attributes: (12/15)
    missing_value:           -9999.0
    NUM_SOIL_LAYERS:         4
    SOIL_LAYER_THICKNESSES:  [ 10.        30.000002  60.000004 100.      ]
    title:                   LIS land surface model output
    institution:             NASA GSFC
    source:                  
    ...                      ...
    comment:                 website: http://lis.gsfc.nasa.gov/
    MAP_PROJECTION:          EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:   -20.975
    SOUTH_WEST_CORNER_LON:   -81.975
    DX:                      0.05
    DY:                      0.05

In [7]:
# Create a list of variables
list_of_variables = ['Rainf_tavg','SoilMoist_inst','Qs_tavg','Evap_tavg','SoilTemp_inst','Qair_f_tavg'] 
print(list_of_variables)
type(list_of_variables)

['Rainf_tavg', 'SoilMoist_inst', 'Qs_tavg', 'Evap_tavg', 'SoilTemp_inst', 'Qair_f_tavg']


list

### Specify ArcGIS project destination

In [9]:
# Define project destination
aprx_path = r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.aprx"

# Open ArcGIS project file
aprx = arcpy.mp.ArcGISProject(aprx_path)
map_name = "Map"
m = aprx.listMaps(map_name)[0]

: 

Looping through list of variables in the list created earlier



In [ ]:
for vars in list_of_variables:
    output_layer_name = "LIS_HIST_2001_to_2023_Monthly_" + vars

    # Import NetCDF as raster layer
    arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster=r"C:\Users\Kris\Documents\amazonforcast\data\malaria_amazon\amazon_monthly_ldas\LIS_HIST_2001_to_2023_Monthly.nc",
    out_multidimensional_raster_layer= output_layer_name,
    variables=vars,
    dimension_def="ALL",
    dimension_ranges=None,
    dimension_values=None,
    dimension="",
    start_of_first_iteration="",
    end_of_first_iteration="",
    iteration_step=None,
    iteration_unit="",
    template='-82 -20.9999999999981 -49.0000000000019 6.00000000000038 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    dimensionless="DIMENSIONS",
    spatial_reference=None
    )

    print(f"Created and saved multidimensional raster layer for variable: {vars}") # Check if the import process is complete

    # Calculate zonal statistic as table by polygon 
    arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.gdb\hybas_sa_lev02_v1c",
    zone_field="PFAF_ID",
    in_value_raster= output_layer_name,
    out_table=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.gdb\ZonalSt_hybas_lev05_" + vars,
    ignore_nodata="DATA",
    statistics_type="MEAN_STD",
    process_as_multidimensional="ALL_SLICES",
    percentile_values=90,
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360,
    out_join_layer="",
    )

    # Join as new 
    arcpy.management.AddJoin(
        
    )

    # Export as new feature class
    arcpy.conversion.ExportFeatures(
    in_features= ,
    out_features=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.gdb\hybas_sa_lev02_v1c_ExportFeatures",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='HYBAS_ID "HYBAS_ID" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,HYBAS_ID,-1,-1;NEXT_DOWN "NEXT_DOWN" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,NEXT_DOWN,-1,-1;NEXT_SINK "NEXT_SINK" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,NEXT_SINK,-1,-1;MAIN_BAS "MAIN_BAS" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,MAIN_BAS,-1,-1;DIST_SINK "DIST_SINK" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,DIST_SINK,-1,-1;DIST_MAIN "DIST_MAIN" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,DIST_MAIN,-1,-1;SUB_AREA "SUB_AREA" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,SUB_AREA,-1,-1;UP_AREA "UP_AREA" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,UP_AREA,-1,-1;PFAF_ID "PFAF_ID" true true false 4 Long 0 0,First,#,hybas_sa_lev02_v1c,PFAF_ID,-1,-1;ENDO "ENDO" true true false 4 Long 0 0,First,#,hybas_sa_lev02_v1c,ENDO,-1,-1;COAST "COAST" true true false 4 Long 0 0,First,#,hybas_sa_lev02_v1c,COAST,-1,-1;ORDER_ "ORDER" true true false 4 Long 0 0,First,#,hybas_sa_lev02_v1c,ORDER_,-1,-1;SORT "SORT" true true false 8 Double 0 0,First,#,hybas_sa_lev02_v1c,SORT,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,hybas_sa_lev02_v1c,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,hybas_sa_lev02_v1c,Shape_Area,-1,-1',
    sort_field=None
    )

    # Remove previous join operations
    arcpy.management.RemoveJoin(
    in_layer_or_view="hybas_sa_lev05_areaofstudy",
    join_name=""
    )


    

In [ ]:

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
    arcpy.management.CreateMosaicDataset(
        in_workspace=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.gdb",
        in_mosaicdataset_name="netcdf_",
        coordinate_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
        num_bands=None,
        pixel_type="",
        product_definition="NONE",
        product_band_definitions=None
    )

In [ ]:
arcpy.management.CreateMosaicDataset(
    in_workspace=r"C:\Users\Kris\Documents\ArcGIS\Projects\bsec_weather_stations\bsec_weather_stations.gdb",
    in_mosaicdataset_name="test",
    coordinate_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None
)


arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.gdb\mosaic_main",
    raster_type=r"NetCDF @ C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.gdb\mosaic_main\RasterType.OBJECTID=1",
    input_path=r"C:\Users\Kris\Documents\amazonforcast\data\malaria_amazon\amazon_monthly_ldas\LIS_HIST_2001_to_2023_Monthly.nc",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="*.nc;*.nc4",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="NO_PYRAMIDS",
    calculate_statistics="CALCULATE_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="ESTIMATE_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\Kris\AppData\Local\ESRI\rasterproxies\mosaic_main"
)

arcpy.management.CopyRaster(
    in_raster=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\amazon_ldas_2001_to_2023.gdb\mosaic_main",
    out_rasterdataset=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\mosaic_main_CopyRaster.crf",
    config_keyword="",
    background_value=None,
    nodata_value="",
    onebit_to_eightbit="NONE",
    colormap_to_RGB="NONE",
    pixel_type="",
    scale_pixel_value="NONE",
    RGB_to_Colormap="NONE",
    format="CRF",
    transform="NONE",
    process_as_multidimensional="ALL_SLICES",
    build_multidimensional_transpose="TRANSPOSE"
)



In [ ]:
def make_netcdf_raster_layer():
    arcpy
    for var in list_of_variables:
        arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=)

        if var = "SoilMoist_tavg" and "SoilTemp_inst":
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=)

if __name__ == '__main__':
    with arcpy.EnvManager()

## Share exported feature layers to ArcGIS Online

### Sign-in to ArcGIS Enterprise

In [1]:
arcpy.SignInToPortal('https://gisanddata.maps.arcgis.com', 'qsu4@jh.edu', '!')


NameError: name 'arcpy' is not defined

### Define Parameters

In [ ]:
feature_layer_path = "path/to/your/feature/layer"
service_name = "MyFeatureLayerService"
service_summary = "Summary of the feature layer service"
service_tags = "tags, for, your, service"

### Define service defination 

### Sign in to ArcGIS Online or ArcGIS Enterprise


In [ ]:
portal_url = 'https://gisanddata.maps.arcgis.com/'  
username = 'qsu4_GISandData'             
password = '!'            

try:
    arcpy.SignInToPortal(portal_url, username, password)
    print("Successfully signed in to the portal.")
except Exception as e:
    print(f"Error signing in to the portal: {e}")


In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023"):
    out_multidimensional_raster = arcpy.ia.GenerateMultidimensionalAnomaly(
        in_multidimensional_raster="mosaic_main",
        variables="evap_tavg",
        method="DIFFERENCE_FROM_MEAN",
        calculation_interval="ALL",
        ignore_nodata="DATA",
        reference_mean_raster=None
    )
    out_multidimensional_raster.save(r"C:\Users\Kris\Documents\ArcGIS\Projects\amazon_ldas_2001_to_2023\mosaic_main_GenerateAnom.crf")